In [1]:
import Pkg
Pkg.add("Pkg")
Pkg.add("Dates")

guypath = "/Users/guyfarmer/ambulance_connect/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/"
michaelpath= "/home/mhilborn/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy"

currentpath = guypath

cd(currentpath)
Pkg.activate(".")

using DataFrames, JLD, Distributions, CSV, Random, Plots,JSON
import DataStructures: PriorityQueue, enqueue!, dequeue!

include(".//src//model.jl")

   Updating registry at `~/.julia/registries/General`
######################################################################### 100.0%
  Resolving package versions...
No Changes to `~/.git/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/test/Project.toml`
Updating `~/.git/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/test/Manifest.toml`
  [f50d1b31] ↑ Rmath_jll v0.2.2+1 ⇒ v0.2.2+2
  Resolving package versions...
No Changes to `~/.git/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/test/Project.toml`
No Changes to `~/.git/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/test/Manifest.toml`
 Activating environment at `~/.git/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Project.toml`
┌ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1278
┌ Info: Precompiling JLD [4138dd39-2aa7-5051-a626-17a0bb65d9c8]
└ @ Base loading.jl:1278
┌ Info: Precompiling Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1278
┌ Info:

reassign_ambulances! (generic function with 2 methods)

In [2]:
#=
Author : Ng Yeesian
Modified : Guy Farmer / Zander Tedjo / Will Worthington / Michael Hilborn
generates the closest dispatch model
=#
mutable struct ClosestDispatch <: DispatchModel
    drivetime::DataFrame
    candidates::Vector{Vector{Int}}
    available::Vector{Int}
    assignment::Vector{Int} # which location the ambulance is assigned to
    ambulances::Vector{Vector{Int}} # list of ambulances assigned to each location
    status::Vector{Symbol} # the current status of the ambulance
        # possible statuses: :available, :responding, :atscene, :conveying, :returning
    fromtime::Vector{Int} # the time it started the new status
    hospital::Vector{Int}
end

function ClosestDispatch(p::DeploymentProblem, drivetime::DataFrame,distribution::Vector{Int})
    candidates = Any[]
    I = 1:p.nlocations
    for region in 1:p.nregions
        push!(candidates, I[vec(p.coverage[region,:])])
    end
    #println("creating the dispatch problem :$distribution")

    assignment = zeros(Int, p.nambulances)
    ambulances = [Int[] for i in 1:p.nlocations]
    k = 1
    for i in eachindex(distribution), j in 1:distribution[i]
        assignment[k] = i
        push!(ambulances[i], k)
        k += 1
    end
    status = fill(:available, p.nambulances)
    fromtime = zeros(Int, p.nambulances)
    hospital = zeros(Int, p.nambulances)
    
    ClosestDispatch(drivetime, candidates, distribution, assignment, ambulances, status, fromtime, hospital)

end

function ClosestDispatch(p::DeploymentProblem, problem::DispatchProblem)
    stn_names = [Symbol("stn$(i)_min") for i in 1:size(p.coverage,2)]
    ClosestDispatch(p, problem.emergency_calls[:, stn_names], problem.available)
end

function update_ambulances!(model::ClosestDispatch, i::Int, delta::Int)
    model.available[i] += delta
    @assert model.available[i] >= 0
end

function closest_available(dispatch::ClosestDispatch, id::Int, problem::DispatchProblem)
    location = 0
    min_time = Inf
    for i in dispatch.candidates[problem.emergency_calls[id, :neighborhood]]
        if problem.available[i] > 0 && dispatch.drivetime[id, i] < min_time
            #println("apparently this station is available : $i and here is the amount of ambulances avalable: $(problem.available[i])")
            location = i
            min_time = dispatch.drivetime[id, i]
        end
    end
    location
end

function respond_to!(dispatch::ClosestDispatch, i::Int, t::Int)
    @assert length(dispatch.ambulances[i]) > 0 "$i: $(dispatch.ambulances[i])"
    amb = popfirst!(dispatch.ambulances[i])
    # @assert redeploy.hospital[amb] == 0
    @assert amb != 0
    @assert dispatch.status[amb] == :available "$amb: $(dispatch.status[amb])"
    dispatch.status[amb] = :responding
    #dispatch.fromtime[amb] = t
    amb
end

function arriveatscene!(dispatch::ClosestDispatch, amb::Int, t::Int)
    @assert dispatch.status[amb] == :responding "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] == 0
    dispatch.status[amb] = :atscene
    dispatch.fromtime[amb] = t
end

function arriveathospital!(dispatch::DispatchModel, amb::Int, hosp::Int, t::Int)
    @assert dispatch.status[amb] == :atscene "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :atHospital
    dispatch.fromtime[amb] = t
    dispatch.hospital[amb] = hosp
end

function returning_to!(dispatch::DispatchModel, amb::Int, t::Int)
    @assert dispatch.status[amb] == :atHospital "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :returning
    dispatch.fromtime[amb] = t
    dispatch.assignment[amb]
end

returning_to! (generic function with 2 methods)

In [11]:
turnaround = Distributions.LogNormal(3.65, 0.3)
lambda = 0
local_path = ""
model_name = "Stochastic"
namb = 40
ncalls = 1000
#using AmbulanceDeployment

pwd()

cd(currentpath * "/test/")
pwd()

#using AmbulanceDeployment
using Dates

#include("..//src//dispatch/closestdispatch.jl")
include("..//src//problem.jl")
include("..//src//evaluate.jl")
#include("..//src//simulate.jl")

hospitals = CSV.File(string(local_path,"../test/austin-data/hospitals.csv")) |> DataFrame
stations = CSV.File(string(local_path,"../test/austin-data/stations.csv")) |> DataFrame
hourly_calls = CSV.File("../test/austin-data/Full_WeekdayCalls.csv") |> DataFrame
adjacent_nbhd = CSV.File("../test/austin-data/adjacent_nbhd.csv") |> DataFrame
coverage = CSV.read("../test/austin-data/coverage_real.csv", DataFrame, header=false)
coverage = convert(Array{Bool, 2}, coverage[:, :])
incidents = CSV.File("../test/austin-data/austin_incidents.csv") |> DataFrame
solverstats = JLD.load("../src/outputs/austin_team_stats_1_03.jld")
amb_deployment = solverstats["amb_deployment"]
model_dict = Dict{String, Symbol}("Stochastic"=>:Stochastic, "Robust01"=>:Robust01, "Robust005"=>:Robust005, "Robust001"=>:Robust001, "Robust0001"=>:Robust0001,
          "Robust00001"=>:Robust00001, "MEXCLP"=>:MEXCLP, "MALP"=>:MALP)


p = DeploymentProblem(
          hourly_calls,
          adjacent_nbhd,
          coverage,
          namb = namb,
          train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
          )
test_calls = CSV.File("../test/austin-data/austin_test_calls.csv")|> DataFrame
test_calls = test_calls[1:ncalls,:]
println("running $namb ambuances & $ncalls calls")

# distribution of ambulances across all of the stations
x = amb_deployment[model_dict[model_name]][namb]

#establishes the dispatch problem
problem = DispatchProblem(test_calls, hospitals, stations, p.coverage, x, turnaround=turnaround)

#establishes the dispatch model
dispatch = ClosestDispatch(p, problem)

Random.seed!(1234)



running 40 ambuances & 1000 calls


MersenneTwister(UInt32[0x000004d2], Random.DSFMT.DSFMT_state(Int32[-1393240018, 1073611148, 45497681, 1072875908, 436273599, 1073674613, -2043716458, 1073445557, -254908435, 1072827086  …  -599655111, 1073144102, 367655457, 1072985259, -1278750689, 1018350124, -597141475, 249849711, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000000000000

In [8]:
using AmbulanceDeployment, Dates

┌ Info: Precompiling AmbulanceDeployment [d2c08a37-75f3-4934-bcbc-64feba624d07]
└ @ Base loading.jl:1278
┌ Warning: Package AmbulanceDeployment does not have Pkg in its dependencies:
│ - If you have AmbulanceDeployment checked out for development and have
│   added Pkg as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with AmbulanceDeployment
└ Loading Pkg into AmbulanceDeployment from project dependency, future warnings for AmbulanceDeployment are suppressed.
   Updating registry at `~/.julia/registries/General`
######################################################################### 100.0%
  Resolving package versions...
No Changes to `~/.git/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Project.toml`
No Changes to `~/.git/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Manifest.toml`
  Resolving package versions...
No Changes to `~/.git/AmbulanceDeployement/AmbulanceDeployment

No Changes to `~/.git/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Project.toml`
No Changes to `~/.git/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Manifest.toml`
  Resolving package versions...
No Changes to `~/.git/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Project.toml`
No Changes to `~/.git/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Manifest.toml`
  Resolving package versions...
No Changes to `~/.git/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Project.toml`
No Changes to `~/.git/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Manifest.toml`
  Resolving package versions...
No Changes to `~/.git/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Project.toml`
No Changes to `~/.git/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Manifest.toml`


In [14]:
Pkg.add("Query")
using Query

struct EMSEngine{T}
    eventlog::DataFrame
    eventqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    shortqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    guiArray::Array{Any, 1}
end

struct gui_event
    #call made / call responded / ambulance arrived / shortfall
    event_type::String
    #neighborhood where call is made from
    neighborhood_id::Int
    # station where call is made from
    deployment_id::Int
    # id of the event generated
    event_id::Int
    ambulance_id::Int
    arrival_time::Int
    remaining_amb::Int
    # current timestamp of event
    timestamp::DateTime
end

## generates the priority queue and instantiates the EMSEngine struct
function generateCalls(problem::DispatchProblem)
    ncalls = nrow(problem.emergency_calls)
    eventlog = DataFrame(
        id = 1:ncalls,
        dispatch_from = zeros(Int, ncalls),
        waittime = fill(0.0, ncalls),
        responsetime = fill(Inf, ncalls),
        scenetime = fill(Inf, ncalls),
        conveytime = fill(Inf, ncalls),
        returntime = fill(Inf, ncalls),
        return_to = zeros(Int, ncalls),
        return_type = fill(:station, ncalls),
        hospital = zeros(Int, ncalls),
        ambulance = zeros(Int, ncalls)
    )
    eventqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    shortqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    guiArray = Any[]
    for i in 1:nrow(problem.emergency_calls)
        t = problem.emergency_calls[i, :arrival_seconds]
        enqueue!(eventqueue, (:call, i, t, problem.emergency_calls[i, :neighborhood]), t)
    end
    EMSEngine{Tuple{Symbol,Int,Int,Int}}(eventlog, eventqueue, shortqueue, guiArray) ## returns the ems engine struct
end

function simulate_events!(problem::DispatchProblem, dispatch::ClosestDispatch)
    engine = generateCalls(problem)
    while(~isempty(engine.eventqueue))
        (event, id, t, value) = dequeue!(engine.eventqueue)

        if event == :call
            # if the event is reachable then change event to station2call
            # if not it is a shortfall
            println("calling event id: $id time: $t value: $value")
            call_event!(engine, problem, dispatch, id, t, value)
           
        elseif event == :station2call
            #once the ambulance arrives at the station change event to call2hospital
            println("Ambulance has arrived on the scene for event $id at time $t")
            station2call_event!(engine, problem, dispatch, id, t, value)
        elseif event == :call2hospital
            #once the ambulance arrives at the hospital it will hand over the patient and then 
            #change the deploy back to its station
            println("Ambulance for event $id has arrived at the hospital at time $t")
            call2hosp_event!(engine, problem, dispatch, id, t, value)

        elseif event == :hospital2station
            #once the ambulance drops off the patient it will reroute to the station
            println("Ambulance for event $id is returning to the station at $t")
            return_event!(engine, problem, dispatch, id, t, value)
            
        elseif event == :done
            #increase the number of ambulances at that station now that it is done
            done_event!(engine, problem, dispatch, id, t, value)
            
        end
    end
    engine.eventlog , engine.guiArray
end


function call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        nbhd::Int; # the neighborhood the call is from
    )

    #check if there is an ambulance within the coverage matrix
    if sum(problem.deployment[problem.coverage[nbhd,:]]) == 0
        @assert false "$id: no ambulance reachable for call at $nbhd"
    #check if one of the ambulances within the coverage is available
    elseif sum(problem.available[problem.coverage[nbhd,:]]) > 0
        i = closest_available(dispatch, id, problem)
        #println("closest available station : $i")
        @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
        update_ambulances!(dispatch, i, -1)
        ems.eventlog[id, :dispatch_from] = i
        @assert problem.available[i] >= 0
        

        travel_time = ceil(Int, 60*2*problem.emergency_calls[id, Symbol("stn$(i)_min")])
        @assert travel_time >= 0
        ems.eventlog[id, :responsetime] = (travel_time + t - problem.emergency_calls[id,:arrival_seconds]) / 60 # minutes

        amb = respond_to!(dispatch, i, t)
        ems.eventlog[id, :ambulance] = amb

        event = gui_event("call responded", problem.emergency_calls[id, :neighborhood], i, id, amb,-1, problem.available[i] ,Dates.now())
        num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
        for i in 1:size(problem.available, 1)
            num_ambulances_array[i] = problem.available[i]
        end
        push!(ems.guiArray,event)
        push!(ems.guiArray,num_ambulances_array)
        enqueue!(ems.eventqueue, (:station2call, id, t + travel_time, amb), t + travel_time)
    #else queue it
    else
        # println(id, ": call from ", nbhd, " queued behind ", problem.wait_queue[nbhd])
        # event = gui_event("call made", nbhd, -1, id, -1,-1,-1,Dates.now())
        # push!(ems.guiArray,event)
        # problem.shortfalls = problem.shortfalls + 1
        # #push!(problem.shortfalls) # count shortfalls
        # push!(problem.wait_queue[nbhd], id) # queue the emergency call
        event = gui_event("shortfall", problem.emergency_calls[id, :neighborhood], -1, id, -1,-1,-1 ,Dates.now())
        push!(ems.guiArray, event)
        enqueue!(ems.shortqueue, (:call, id, t, problem.emergency_calls[id, :neighborhood]), t)
        println(id, ": call from ", nbhd, " is a shortfall")
    end
end

function station2call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    arriveatscene!(dispatch, amb, t)
    # time the ambulance spends at the scene
    scene_time = ceil(Int,60*0.4*rand(problem.turnaround)) # 60sec*0.4*mean(40minutes) ~ 15minutes
    ems.eventlog[id, :scenetime] = scene_time / 60 # minutes
    @assert scene_time > 0
    
    event = gui_event("ambulance arrived", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    enqueue!(ems.eventqueue, (:call2hospital, id, t + scene_time, amb), t + scene_time)
end

function call2hosp_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    h = let mintime = Inf, minindex = 0
        for h in 1:nrow(problem.hospitals)
            traveltime = problem.emergency_calls[id, Symbol("hosp$(h)_min")]
            if !isna(traveltime) && !(traveltime=="NA") && traveltime < mintime
                @assert traveltime >= 0
                minindex = h; mintime = traveltime
            end
        end
        minindex
    end
    @assert h != 0
    dispatch.hospital[amb] = ems.eventlog[id, :hospital] = h
    conveytime = 60*15 + ceil(Int, 60*problem.emergency_calls[id, Symbol("hosp$(h)_min")]) # ~20minutes
    ems.eventlog[id, :conveytime] = conveytime / 60 # minutes
    @assert conveytime >= 0 conveytime
    arriveathospital!(dispatch, amb, h, t)
    
    #         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    event = gui_event("ambulance hospital", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    
    enqueue!(ems.eventqueue, (:hospital2station, id, t+conveytime, amb), t+conveytime)
end

function return_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    stn = returning_to!(dispatch, amb, t)
    h = dispatch.hospital[amb]
    returntime = ceil(Int,60*2*problem.hospitals[h, Symbol("stn$(stn)_min")]) # ~ 10minutes
    dispatch.hospital[amb] = 0
    ems.eventlog[id, :returntime] = returntime / 60 # minutes
    @assert returntime >= 0 returntime
    t_end = t + returntime
    
#         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    
    event = gui_event("ambulance returning", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    
    enqueue!(ems.eventqueue, (:done, id, t_end, amb), t_end)
    # num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
    # for i in 1:size(problem.available, 1)
    #     num_ambulances_array[i] = problem.available[i]
    # end
    # push!(ems.guiArray,num_ambulances_array)
end

function done_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    i = dispatch.assignment[amb]
    println("Ambulance from event $id has returned to station $i")
    @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
    update_ambulances!(dispatch, i, 1)
    push!(dispatch.ambulances[i], amb)
    dispatch.status[amb] = :available
    ems.eventlog[id, :return_to] = id
    @assert problem.available[i] > 0
    
#         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    
    event = gui_event("ambulance returned", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    while(~isempty(ems.shortqueue))
        # record time difference
        (event, short_id, short_t, value) = dequeue!(ems.shortqueue)
        enqueue!(ems.eventqueue, (event, short_id, t , value), short_t)
    end
end


  Resolving package versions...
No Changes to `~/.git/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Project.toml`
No Changes to `~/.git/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Manifest.toml`


done_event! (generic function with 1 method)

In [15]:
@time df, guiArray = simulate_events!(problem, dispatch)

calling event id: 1 time: 38 value: 83
calling event id: 2 time: 575 value: 38
Ambulance has arrived on the scene for event 1 at time 638
calling event id: 3 time: 673 value: 129
calling event id: 4 time: 691 value: 107
calling event id: 5 time: 1166 value: 113
calling event id: 6 time: 1187 value: 51
Ambulance for event 1 has arrived at the hospital at time 1321
calling event id: 7 time: 1450 value: 76
calling event id: 8 time: 1578 value: 67
calling event id: 9 time: 1640 value: 52
Ambulance has arrived on the scene for event 6 at time 1787
calling event id: 10 time: 1838 value: 67
calling event id: 11 time: 1899 value: 67
calling event id: 12 time: 1901 value: 82
Ambulance has arrived on the scene for event 4 at time 1912
Ambulance has arrived on the scene for event 2 at time 1924
calling event id: 13 time: 2091 value: 98
calling event id: 14 time: 2117 value: 113
Ambulance has arrived on the scene for event 3 at time 2140
Ambulance has arrived on the scene for event 8 at time 2178


Ambulance for event 39 has arrived at the hospital at time 12639
Ambulance has arrived on the scene for event 43 at time 12660
Ambulance from event 33 has returned to station 5
calling event id: 45 time: 12678 value: 99
Ambulance has arrived on the scene for event 44 at time 12679
Ambulance for event 36 is returning to the station at 12822
calling event id: 46 time: 12885 value: 124
Ambulance for event 38 has arrived at the hospital at time 12950
Ambulance for event 35 is returning to the station at 13047
Ambulance for event 37 has arrived at the hospital at time 13064
calling event id: 47 time: 13075 value: 144
Ambulance for event 42 has arrived at the hospital at time 13208
Ambulance from event 21 has returned to station 31
Ambulance for event 40 has arrived at the hospital at time 13278
Ambulance has arrived on the scene for event 45 at time 13278
Ambulance from event 35 has returned to station 17
Ambulance for event 34 is returning to the station at 13470
Ambulance has arrived on t

Ambulance from event 70 has returned to station 15
Ambulance from event 69 has returned to station 8
Ambulance from event 72 has returned to station 5
Ambulance has arrived on the scene for event 77 at time 25686
Ambulance from event 71 has returned to station 1
calling event id: 80 time: 25961 value: 52
Ambulance has arrived on the scene for event 78 at time 26160
Ambulance for event 74 is returning to the station at 26403
Ambulance for event 75 is returning to the station at 26486
Ambulance for event 77 has arrived at the hospital at time 26507
Ambulance for event 76 has arrived at the hospital at time 26530
calling event id: 81 time: 26594 value: 114
calling event id: 82 time: 26642 value: 98
calling event id: 83 time: 26657 value: 115
Ambulance has arrived on the scene for event 79 at time 26662
Ambulance from event 67 has returned to station 2
Ambulance from event 73 has returned to station 28
calling event id: 84 time: 26802 value: 98
Ambulance from event 74 has returned to stati

Ambulance has arrived on the scene for event 119 at time 45031
Ambulance for event 112 is returning to the station at 45088
calling event id: 123 time: 45101 value: 108
Ambulance has arrived on the scene for event 117 at time 45128
Ambulance for event 115 has arrived at the hospital at time 45172
Ambulance has arrived on the scene for event 118 at time 45196
calling event id: 124 time: 45277 value: 67
calling event id: 125 time: 45304 value: 109
calling event id: 126 time: 45420 value: 143
Ambulance has arrived on the scene for event 120 at time 45575
Ambulance has arrived on the scene for event 122 at time 45607
Ambulance has arrived on the scene for event 123 at time 45701
Ambulance has arrived on the scene for event 121 at time 45766
calling event id: 127 time: 45780 value: 97
Ambulance for event 114 is returning to the station at 45879
Ambulance for event 118 has arrived at the hospital at time 45896
Ambulance from event 113 has returned to station 25
Ambulance for event 117 has ar

Ambulance has arrived on the scene for event 149 at time 52909
Ambulance for event 138 is returning to the station at 52934
calling event id: 154 time: 53031 value: 83
Ambulance has arrived on the scene for event 150 at time 53047
Ambulance for event 146 has arrived at the hospital at time 53107
calling event id: 155 time: 53166 value: 128
Ambulance for event 148 has arrived at the hospital at time 53177
Ambulance for event 143 has arrived at the hospital at time 53182
calling event id: 156 time: 53203 value: 98
calling event id: 157 time: 53232 value: 65
Ambulance from event 138 has returned to station 8
Ambulance from event 137 has returned to station 17
Ambulance for event 144 has arrived at the hospital at time 53339
calling event id: 158 time: 53375 value: 128
calling event id: 159 time: 53522 value: 98
Ambulance for event 139 is returning to the station at 53531
Ambulance from event 135 has returned to station 14
Ambulance for event 152 has arrived at the hospital at time 53643
c

...

946: call from 113 is a shortfall
calling event id: 937 time: 263100 value: 129
937: call from 129 is a shortfall
Ambulance has arrived on the scene for event 940 at time 263116
Ambulance from event 905 has returned to station 29
calling event id: 933 time: 263170 value: 116
933: call from 116 is a shortfall
calling event id: 937 time: 263170 value: 129
937: call from 129 is a shortfall
calling event id: 946 time: 263170 value: 113
946: call from 113 is a shortfall
Ambulance has arrived on the scene for event 939 at time 263170
Ambulance from event 920 has returned to station 1
calling event id: 933 time: 263205 value: 116
933: call from 116 is a shortfall
calling event id: 946 time: 263205 value: 113
calling event id: 937 time: 263205 value: 129
937: call from 129 is a shortfall
Ambulance from event 928 has returned to station 3
calling event id: 933 time: 263231 value: 116
933: call from 116 is a shortfall
calling event id: 937 time: 263231 value: 129
937: call from 129 is a shortfal

956: call from 128 is a shortfall
Ambulance from event 929 has returned to station 4
calling event id: 957 time: 266331 value: 40
957: call from 40 is a shortfall
calling event id: 956 time: 266331 value: 128
956: call from 128 is a shortfall
calling event id: 955 time: 266331 value: 98
Ambulance from event 940 has returned to station 15
calling event id: 957 time: 266377 value: 40
957: call from 40 is a shortfall
calling event id: 956 time: 266377 value: 128
Ambulance for event 951 has arrived at the hospital at time 266400
calling event id: 961 time: 266455 value: 142
Ambulance from event 934 has returned to station 25
calling event id: 957 time: 266464 value: 40
957: call from 40 is a shortfall
calling event id: 962 time: 266517 value: 68
962: call from 68 is a shortfall
Ambulance from event 942 has returned to station 22
calling event id: 957 time: 266642 value: 40
957: call from 40 is a shortfall
calling event id: 962 time: 266642 value: 68
962: call from 68 is a shortfall
Ambulan

Ambulance from event 952 has returned to station 17
calling event id: 970 time: 270389 value: 52
970: call from 52 is a shortfall
Ambulance from event 960 has returned to station 19
calling event id: 970 time: 270401 value: 52
Ambulance from event 958 has returned to station 22
Ambulance for event 966 is returning to the station at 270530
calling event id: 974 time: 270537 value: 98
Ambulance has arrived on the scene for event 957 at time 270630
Ambulance for event 965 is returning to the station at 270730
Ambulance from event 961 has returned to station 26
Ambulance for event 972 has arrived at the hospital at time 270910
Ambulance for event 973 has arrived at the hospital at time 270974
Ambulance from event 955 has returned to station 4
Ambulance from event 956 has returned to station 15
Ambulance from event 959 has returned to station 8
Ambulance for event 967 has arrived at the hospital at time 271158
Ambulance for event 957 has arrived at the hospital at time 271184
Ambulance has 

  8.345848 seconds (600.37 k allocations: 22.098 MiB)


(1000×11 DataFrame. Omitted printing of 6 columns
│ Row  │ id    │ dispatch_from │ waittime │ responsetime │ scenetime │
│      │ Int64 │ Int64         │ Float64  │ Float64      │ Float64   │
├──────┼───────┼───────────────┼──────────┼──────────────┼───────────┤
│ 1    │ 1     │ 1             │ 0.0      │ 10.0         │ 11.3833   │
│ 2    │ 2     │ 43            │ 0.0      │ 22.4833      │ 13.0      │
│ 3    │ 3     │ 14            │ 0.0      │ 24.45        │ 16.25     │
│ 4    │ 4     │ 34            │ 0.0      │ 20.35        │ 12.2667   │
│ 5    │ 5     │ 17            │ 0.0      │ 19.5333      │ 19.8333   │
│ 6    │ 6     │ 22            │ 0.0      │ 10.0         │ 13.35     │
│ 7    │ 7     │ 39            │ 0.0      │ 49.0833      │ 12.1333   │
│ 8    │ 8     │ 8             │ 0.0      │ 10.0         │ 11.25     │
│ 9    │ 9     │ 8             │ 0.0      │ 22.7         │ 9.08333   │
│ 10   │ 10    │ 3             │ 0.0      │ 19.2333      │ 12.75     │
⋮
│ 990  │ 990   │ 44      

In [16]:
guiArray

7280-element Array{Any,1}:
 gui_event("call responded", 83, 1, 1, 1, -1, 0, DateTime("2021-03-31T15:19:28.245"))
 Integer[0, 2, 1, 1, 1, 0, 1, 2, 2, 0  …  0, 0, 0, 0, 1, 0, 1, 0, 1, 1]
 gui_event("call responded", 38, 43, 2, 39, -1, 0, DateTime("2021-03-31T15:19:28.245"))
 Integer[0, 2, 1, 1, 1, 0, 1, 2, 2, 0  …  0, 0, 0, 0, 1, 0, 1, 0, 0, 1]
 gui_event("ambulance arrived", 83, -1, 1, 1, 638, -1, DateTime("2021-03-31T15:19:28.246"))
 gui_event("call responded", 129, 14, 3, 13, -1, 0, DateTime("2021-03-31T15:19:28.246"))
 Integer[0, 2, 1, 1, 1, 0, 1, 2, 2, 0  …  0, 0, 0, 0, 1, 0, 1, 0, 0, 1]
 gui_event("call responded", 107, 34, 4, 36, -1, 1, DateTime("2021-03-31T15:19:28.247"))
 Integer[0, 2, 1, 1, 1, 0, 1, 2, 2, 0  …  0, 0, 0, 0, 1, 0, 1, 0, 0, 1]
 gui_event("call responded", 113, 17, 5, 19, -1, 3, DateTime("2021-03-31T15:19:28.247"))
 Integer[0, 2, 1, 1, 1, 0, 1, 2, 2, 0  …  0, 0, 0, 0, 1, 0, 1, 0, 0, 1]
 gui_event("call responded", 51, 22, 6, 24, -1, 1, DateTime("2021-03-31T15:19:28